In [ ]:
# import pytroch geometric hetero data
# from torch_geometric.data import HeteroData
# import torch
# data = HeteroData.from_dict(torch.load('HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'))

In [7]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [8]:
from learnings_sampler_v1 import get_datasets

In [9]:
train_data, val_data, test_data = get_datasets(get_edge_attr=False,filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50)

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


In [ ]:
# people have jobs, if multiple, we average
# jobs have skills, if multiple, we average
#  

# so row is [pid, lid][person learning person_features person_jobs job_skills target_watched]

In [5]:
import torch
from tqdm.auto import tqdm
from torch_geometric.data import HeteroData
import os 

def make_ds(data, split_name):
    def skills_avg(edge_index_to_skill, entity_id):
        # eg job->skill, entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        if torch.sum(mask1)==0:
            return torch.zeros(1, data['skills'].x.shape[1])
        a = data['skills'].x[mask2]
        if a.dim() == 1:
            return a.unsqueeze(0)
        else:
            return a.mean(axis=0).squeeze().unsqueeze(0)

    
    job_skills = []
    for job_id in tqdm(range(data['jobs'].num_nodes)):
        e= skills_avg(data['jobs', 'rev_job_skill', 'skills'].edge_index,job_id)
        job = data['jobs'].x[job_id,:].unsqueeze(0)
        
        job_skills.append(torch.cat((job,e),dim=1))
    
    learning_skills = []
    for learning_id in tqdm(range(data['courses_and_programs'].num_nodes)):
        e  = skills_avg(data['courses_and_programs', 'rev_course_and_program_skill', 'skills'].edge_index,learning_id)
        learning = data['courses_and_programs'].x[learning_id,:].unsqueeze(0)
        learning_skills.append(torch.cat((learning,e),dim=1))
      
        
        
    jobs_skills = torch.cat((job_skills),dim=0)
    
    #torch.save(torch.cat((learning_skills),dim=0),ROOT_FOLDER+'learnings/'+split_name+'/learnings.pt')
    #del learning_skills

    def avg_of_computed(edge_index_to_skill, opposite_entity_id, opposite_datax):
        # eg job->skill, oposite_entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==opposite_entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        if torch.sum(mask1)==0:
            return torch.zeros(1, opposite_datax.shape[1])
        a = opposite_datax[mask2]
        if a.dim() == 1:
            return a.unsqueeze(0)
        else:
            return a.mean(axis=0).squeeze().unsqueeze(0)
    
    def avg_onehot_of_computed(edge_index_to_skill, opposite_entity_id, opposite_name):
        # eg job->skill, oposite_entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==opposite_entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        #onehot = torch.zeros(1,data[opposite_name].num_nodes)
        if mask2.numel()==0:
            return torch.tensor([[-1]])
        elif mask2.dim()==0:
            return torch.tensor([[mask2.item()]])
        else:
            return torch.tensor([[mask2[0].item()]])

    
    

    persons = []


    

    
    for person_i in tqdm(range(data['people'].x.shape[0])):
        pid = person_i 
        supervisor = avg_onehot_of_computed(data['people','supervisor_supervisee','people'].edge_index,person_i,'people')
        organization = avg_onehot_of_computed(data['people', 'organization_student', 'organizations'].edge_index,person_i,'organizations')

        jobs = avg_of_computed(data['people','rev_job_student','jobs'].edge_index, person_i, jobs_skills)
        person = data['people'].x[person_i,:].unsqueeze(0)

        persons.append(torch.cat((person,jobs,supervisor,organization),dim=1)) # 
    
    persons = torch.cat((persons),dim=0)
    
    
    dataset = HeteroData()
    dataset['people'].x = persons
    dataset['courses_and_programs'].x = learning_skills
    dataset['people', 'completed','courses_and_programs'].edge_index = data['people', 'rev_course_and_programs_student', 'courses_and_programs'].edge_index
    a = data['courses_and_programs','course_and_programs_student','people'].edge_label_index
    
    dataset['people', 'completed','courses_and_programs'].edge_label_index = torch.cat((a[1,:].unsqueeze(0),a[0,:].unsqueeze(0)),dim=0)
    



    filename = ROOT_FOLDER+f'FactorizationMachines_Dataset_{split_name}_v1.pt'
    if os.path.exists(filename):
        raise Exception('File already exists')
    else:
        torch.save(dataset.to_dict(), filename)

In [ ]:
make_ds(train_data,'train')
make_ds(val_data, 'val')
make_ds(test_data, 'test') 

In [ ]:
# sampler

def nf_sampler(batch_size, neg_sample_ratio=1, edge_label_index, total_num_target_nodes, num_organizations):
    # triplet mode only
    # sample some random edges
    num_samples = batch_size
    sampled_indices = torch.randint(0, edge_label_index.shape[1], (num_samples,))
    sampled_edges = edge_label_index[:, sampled_indices]

    neg_samples = batch_size*neg_sample_ratio
    
    s = sampled_edges[0,:].unsqueeze(0)
    src_edges = s
    for i in range(neg_sample_ratio-1):
        src_edges= torch.cat((src_edges,s),dim=1)

    sampled_negatives = torch.randint(0, num_organizations, (neg_samples,)).squeeze().unsqueeze(0)
   

    negative_edge_label_index = torch.cat((src_edges, sampled_negatives),dim=0)
    edge_label_indices = torch.cat((sampled_edges, negative_edge_label_index),dim=1)
    
    return edge_label_indices, torch.cat((torch.ones(batch_size), torch.zeros(neg_samples)))
    
    
    

In [1]:
test_data

NameError: name 'test_data' is not defined

In [23]:
# this makes label encoded dataset
import torch
from tqdm.auto import tqdm
from torch_geometric.data import HeteroData
import os 

def make_ds_labelencoding(data, split_name):
    def get_label_encoding(edge_index_to_skill, entity_id, first_n):
        # eg job->skill, entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        
        # sort mask2 by its values
        if mask2.numel()==0:
            return torch.zeros(first_n, dtype=torch.long).unsqueeze(0)
        elif mask2.dim()==0:
            a = torch.zeros(first_n, dtype=torch.long)
            a[0] = mask2.item()+1
            return a.unsqueeze(0)
        
        mask2 = mask2[torch.argsort(mask2)][:first_n]
        
        # create a Long Tensor of length 50 with zero values
        a = torch.zeros(first_n, dtype=torch.long)
        # fill the first values with the mask2 values
        a[:mask2.shape[0]] = mask2+1
        return a.unsqueeze(0)

    
    job_skills = []
    for job_id in tqdm(range(data['jobs'].num_nodes)):
        e= get_label_encoding(data['jobs', 'rev_job_skill', 'skills'].edge_index,job_id, 50)
        job_id = torch.tensor([[job_id]])
       
        job_skills.append(torch.cat((job_id,e),dim=1))
    
    learning_skills = []
    for learning_id in tqdm(range(data['courses_and_programs'].num_nodes)):
        e  = get_label_encoding(data['courses_and_programs', 'rev_course_and_program_skill', 'skills'].edge_index,learning_id, 25)
        learning_id = torch.tensor([[learning_id]])
        learning_skills.append(torch.cat((learning_id,e),dim=1))

        
        
    jobs_skills = torch.cat((job_skills),dim=0)
    learning_skills  = torch.cat((learning_skills),dim=0)
    learning_skills = learning_skills.long()
    
    #torch.save(torch.cat((learning_skills),dim=0),ROOT_FOLDER+'learnings/'+split_name+'/learnings.pt')
    #del learning_skills

    # def avg_of_computed(edge_index_to_skill, opposite_entity_id, first_n):
    #     # eg job->skill, oposite_entity_id = job 254
    #     mask1 = edge_index_to_skill[0,:]==opposite_entity_id
    #     mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        
    #     # sort mask2 by its values
    #     mask2 = mask2[torch.argsort(mask2)][:3]
        
    #     # create a Long Tensor of length 50 with zero values
    #     a = torch.zeros(first_n, dtype=torch.long)
    #     # fill the first values with the mask2 values
    #     a[:mask2.shape[0]] = mask2+1
    #     return a
  
    
    def get_job(edge_index_to_skill, opposite_entity_id, datax):
        # eg job->skill, oposite_entity_id = job 254
        mask1 = edge_index_to_skill[0,:]==opposite_entity_id
        mask2 = edge_index_to_skill[1,:][mask1].squeeze()
        #onehot = torch.zeros(1,data[opposite_name].num_nodes)
        if mask2.numel()==0:
            return torch.tensor([[0]])
        elif mask2.dim()==0:
            return datax[mask2.item()]
        else:
            return datax[mask2[0].item()]
       
    
    person_dim = 1
    jobs_dim = 1
    skills_dim = 50
    supervisor_dim = 2
    organization_dim = 2
    
    print(learning_skills.shape)
    
    num_persons= test_data['people'].x.shape[0]
    num_jobs = test_data['jobs'].x.shape[0]
    num_skills = test_data['skills'].x.shape[0]
    num_supervisors = num_persons
    num_organizations = test_data['organizations'].x.shape[0]
    
    
    # 1 person, 1 job, 50 skill, 2 supervisor, 2 organization
    labelencoding_person = torch.zeros((person_dim+jobs_dim+skills_dim+supervisor_dim+organization_dim))
    
    # make label encoding vector which has the number of each entity for each dimension
    # make a long tensor of size (num_persons, person_dim+jobs_dim+supervisor_dim+organization_dim)
    
    labelencoding_person[:person_dim] = num_persons
    labelencoding_person[person_dim:person_dim+jobs_dim] = num_jobs
    labelencoding_person[person_dim+jobs_dim:person_dim+jobs_dim+skills_dim] = num_skills
    labelencoding_person[person_dim+jobs_dim+skills_dim:person_dim+jobs_dim+skills_dim+supervisor_dim] = num_supervisors
    labelencoding_person[person_dim+jobs_dim+skills_dim+supervisor_dim:] = num_organizations
    print(labelencoding_person,labelencoding_person.shape)
    learnings_dim = 1
    skills_dim2 = 25
    num_learnings = test_data['courses_and_programs'].x.shape[0]
    labelencoding_learnings = torch.zeros((learnings_dim+skills_dim2))
    labelencoding_learnings[:learnings_dim] = num_learnings
    labelencoding_learnings[learnings_dim:] = num_skills
    print(labelencoding_learnings,labelencoding_learnings.shape)
    persons = torch.zeros((data['people'].x.shape[0], person_dim+jobs_dim+skills_dim+supervisor_dim+organization_dim))
    xx=True
    for person_i in tqdm(range(data['people'].x.shape[0])):
        supervisor = get_label_encoding(data['people','supervisor_supervisee','people'].edge_index, person_i, supervisor_dim)
        organization = get_label_encoding(data['people', 'organization_student', 'organizations'].edge_index, person_i, organization_dim)

        jobs = get_job(data['people','rev_job_student','jobs'].edge_index, person_i, jobs_skills)
        person = torch.LongTensor([[person_i]])

        # Assign each feature to the corresponding row in the tensor
        persons[person_i, :person_dim] = person
        persons[person_i, person_dim:person_dim+jobs_dim+skills_dim] = jobs
        persons[person_i, person_dim+jobs_dim+skills_dim:person_dim+jobs_dim+skills_dim+supervisor_dim] = supervisor
        persons[person_i, person_dim+jobs_dim+skills_dim+supervisor_dim:] = organization
        
        if xx:
            xx=False
            print(persons.shape)

    print('Filled tensor')

    
    
    dataset = HeteroData()
    print('persons')
    dataset['people'].x = persons.long()
    dataset['people'].labelencoding = labelencoding_person.long()
    dataset['courses_and_programs'].x = learning_skills.long()
    dataset['courses_and_programs'].labelencoding = labelencoding_learnings.long()
    dataset['people', 'completed','courses_and_programs'].edge_index = data['people', 'rev_course_and_programs_student', 'courses_and_programs'].edge_index
    a = data['courses_and_programs','course_and_programs_student','people'].edge_label_index
    
    dataset['people', 'completed','courses_and_programs'].edge_label_index = torch.cat((a[1,:].unsqueeze(0),a[0,:].unsqueeze(0)),dim=0)
    

    filename = ROOT_FOLDER+f'FactorizationMachines_Dataset_{split_name}_labelencoded_v2.pt'
    if os.path.exists(filename):
        raise Exception('File already exists')
    else:
        torch.save(dataset.to_dict(), filename)

make_ds_labelencoding(train_data,'train')
make_ds_labelencoding(val_data,'val')
make_ds_labelencoding(test_data,'test')

  0%|          | 0/55638 [00:00<?, ?it/s]

100%|██████████| 55796/55796 [00:13<00:00, 4131.37it/s]


torch.Size([55796, 26])
tensor([293444.,  55638., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 293444., 293444.,  13613.,  13613.]) torch.Size([56])
tensor([ 55796., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052.]) torch.Size([26])


  0%|          | 192/293444 [00:00<02:32, 1918.57it/s]

torch.Size([293444, 56])


...

torch.Size([55796, 26])
tensor([293444.,  55638., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 293444., 293444.,  13613.,  13613.]) torch.Size([56])
tensor([ 55796., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052., 264052., 264052., 264052., 264052., 264052., 264052.,
        264052., 264052.]) torch.Size([26])


  0%|          | 107/293444 [00:00<04:35, 1063.08it/s]

torch.Size([293444, 56])


100%|██████████| 293444/293444 [04:00<00:00, 1219.64it/s]


Filled tensor
persons


In [16]:
person_dim = 1
jobs_dim = 51
supervisor_dim = 2
organization_dim = 2

num_persons= test_data['people'].x.shape[0]
num_jobs = test_data['jobs'].x.shape[0]
num_supervisors = num_persons
num_organizations = test_data['organizations'].x.shape[0]



# make label encoding vector which has the number of each entity for each dimension
# make a long tensor of size (num_persons, person_dim+jobs_dim+supervisor_dim+organization_dim)
labelencoding_person = torch.zeros((person_dim+jobs_dim+supervisor_dim+organization_dim))
labelencoding_person[:person_dim] = num_persons
labelencoding_person[person_dim:person_dim+jobs_dim] = num_jobs
labelencoding_person[person_dim+jobs_dim:person_dim+jobs_dim+supervisor_dim] = num_supervisors
labelencoding_person[person_dim+jobs_dim+supervisor_dim:] = num_organizations

learnings_dim = 1
skills_dim = 50
num_skills = test_data['skills'].x.shape[0]
num_learnings = test_data['courses_and_programs'].x.shape[0]
labelencoding_learnings = torch.zeros((learnings_dim+skills_dim))
labelencoding_learnings[:learnings_dim] = num_learnings
labelencoding_learnings[learnings_dim:] = num_skills

In [26]:
ROOT_FOLDER+f'FactorizationMachines_Dataset_{"train"}_labelencoded_v1.pt'

'FactorizationMachines_Dataset_train_labelencoded_v1.pt'

In [27]:
data = HeteroData.from_dict(torch.load(ROOT_FOLDER+f'FactorizationMachines_Dataset_{"train"}_labelencoded_v1.pt'))

In [32]:
data['people'].x.dtype

torch.float32